In [2]:
from abc import ABC, abstractmethod

class Runnables(ABC):

    @abstractmethod
    def invoke(input_data):
        pass

In [5]:
import random

class DummyLLM(Runnables):

    def __init__(self):
        print("LLM Created")
    
    def invoke(self,prompt):
        return self.predict(prompt);

    def predict(self, prompt):
        response_list = ["AI stands for Artifical Intelligence",
        "ML stands for machine learning",
        "DL stands for Deep Learning"]

        return {"response":random.choice(response_list)}

In [7]:
class DummyPromptTemplate(Runnables):

    def __init__(self, template, inputvariables):
        self.template = template
        self.inputvariables = inputvariables
    
    def invoke(self, inputvariables):
        return self.template.format(**inputvariables)
    
    def format(self, inputvariables):
        return self.template.format(**inputvariables)

In [11]:
class DummyStrOutputParser(Runnables):

    def invoke(self,input_data):
        return input_data["response"]

In [8]:
class RunnableConnector(Runnables):

    def __init__(self, runnable_list):
        self.runnable_list = runnable_list
    
    def invoke(self, input_data):
        for task in self.runnable_list:
            input_data = task.invoke(input_data)

        return input_data


In [12]:
prompt = DummyPromptTemplate(template="Write a poem on topic {topic}", inputvariables=["topic"])
llm = DummyLLM()
parser = DummyStrOutputParser()
chain = RunnableConnector([prompt, llm, parser])
chain.invoke({"topic": "AI"})

LLM Created


'AI stands for Artifical Intelligence'

In [13]:
DummyTemplate1 = DummyPromptTemplate(template="Write a poem about topic {topic}", inputvariables=["topic"])
DummyTemplate2 = DummyPromptTemplate(template="Summarize the topic {response}", inputvariables=["response"])

result = RunnableConnector([DummyTemplate1, llm, DummyTemplate2, llm, parser]).invoke({"topic": "AI"})
print(result)

AI stands for Artifical Intelligence


In [17]:
DummyTemplate1 = DummyPromptTemplate(template="Write a poem about topic {topic}", inputvariables=["topic"])
DummyTemplate2 = DummyPromptTemplate(template="Summarize the topic {response}", inputvariables=["response"])
chain1 = RunnableConnector([DummyTemplate1, llm])
chain2 = RunnableConnector([DummyTemplate2, llm, parser])
final_chain = RunnableConnector([chain1, chain2])
result = final_chain.invoke({"topic": "AI"})
print(result)

DL stands for Deep Learning
